<img src="sztaki_logo.jpg" height="112" width="400" align="left"><br>
<br>

# Test Spark Cluster

### Import python libraries

In [3]:
from pyspark import SparkContext, SparkConf

In [4]:
SparkMasterIP="xxxSPARKMASTERIPxxx"

### Start Spark Application

In [5]:
# Start Spark local mode
# sc = SparkContext(appName="test", master="local")

# Start Spark cluster mode

sc = SparkContext(appName="test", master="spark://"+SparkMasterIP+":7077")


In [6]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Check Spark Context

In [7]:
sc

<SparkContext master=spark://193.224.59.248:7077 appName=test>

### Ohter python libraries

In [8]:
from pyspark.sql import Row
from pyspark.sql import SparkSession

### Create an RDD and fill in a series of data

In [9]:
rdd = sc.parallelize(\
    [Row(name='Steve', age=40, id=1),\
     Row(name='Lui', age=10, id=2),\
     Row(name='Mike', age=99, id=3)\
    ]\
    , numSlices=3
)

### Unit test

In [10]:
# Test that the number of rows in resilient distributed dateset is equal 3

In [11]:
assert rdd.count() == 3, "Should be 3"

### Unit test

In [12]:
# Test that rdd object hasn't got a methond named toDF

In [13]:
assert hasattr(rdd, "toDF") == False, "Should be False"

In [14]:
hasattr(rdd, "toDF")
## False

False

### Create Spark Session

In [15]:
spark = SparkSession(sc)

In [16]:
assert hasattr(rdd, "toDF") == True, "Should be True"

In [17]:
hasattr(rdd, "toDF")
## True

True

### Create a DataFrame from RDD

In [18]:
dataFrame = rdd.toDF()

### Data in RDD

In [19]:
rdd.toDF().show()

+---+---+-----+
|age| id| name|
+---+---+-----+
| 40|  1|Steve|
| 10|  2|  Lui|
| 99|  3| Mike|
+---+---+-----+



### Data in DataFrame

In [20]:
dataFrame.show()

+---+---+-----+
|age| id| name|
+---+---+-----+
| 40|  1|Steve|
| 10|  2|  Lui|
| 99|  3| Mike|
+---+---+-----+



### Get the number of Partitions

In [21]:
dataFrame.rdd.getNumPartitions()

# must be 3

3

### Test

In [22]:
if( dataFrame.first().index('Steve') == 2):
    Test = "Spark and Python program successfully completed."
else:
    Test = "Something goes wrong."

### Stop Spark Application

In [23]:
sc.stop()
spark.stop()

### Result

In [24]:
print(Test)

Spark and Python program successfully completed.


# Test HDFS

In [25]:
!wget https://raw.githubusercontent.com/occopus/docs/master/sphinx/source/tutorial-bigdata-ai.rst -O /home/sparkuser/text.txt

--2020-08-18 11:08:48--  https://raw.githubusercontent.com/occopus/docs/master/sphinx/source/tutorial-bigdata-ai.rst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44068 (43K) [text/plain]
Saving to: ‘/home/sparkuser/text.txt’

/home/sparkuser/tex 100%[===================>]  43.04K  --.-KB/s    in 0.009s  

2020-08-18 11:08:48 (4.75 MB/s) - ‘/home/sparkuser/text.txt’ saved [44068/44068]



In [26]:
!/home/sparkuser/hadoop/bin/hdfs dfs -put /home/sparkuser/text.txt /

In [27]:
sc = SparkContext(appName="test", master="spark://"+SparkMasterIP+":7077")
distFile = sc.textFile("hdfs://"+SparkMasterIP+":9000/text.txt")

In [28]:
nonempty_lines = distFile.filter(lambda x: len(x) > 0)
print('Nonempty lines', nonempty_lines.count())

Nonempty lines 475


In [29]:
words = nonempty_lines.flatMap(lambda x: x.split(' '))

In [30]:
wordcounts = words.map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x, y: x+y) \
                  .map(lambda x: (x[1], x[0])).sortByKey(False)
print('Top 100 words:')
print(wordcounts.take(100))

Top 100 words:
[(2087, ''), (421, 'the'), (123, 'and'), (111, 'of'), (101, 'to'), (98, 'for'), (85, 'Spark'), (84, 'a'), (81, 'is'), (78, '..'), (71, '-'), (63, 'in'), (62, 'you'), (58, 'can'), (57, '#.'), (43, 'with'), (42, 'on'), (40, 'your'), (39, 'node'), (38, 'nodes'), (35, 'be'), (34, 'infrastructure'), (32, 'cloud'), (32, 'code::'), (30, 'resource'), (30, 'port'), (30, 'plugin'), (28, 'this'), (28, 'sure'), (26, 'The'), (26, 'by'), (25, 'You'), (25, 'bash'), (24, 'are'), (24, 'TCP'), (24, 'authentication'), (23, 'using'), (23, 'we'), (23, 'set'), (22, 'an'), (22, 'Occopus'), (22, 'important::'), (21, 'Apache'), (21, 'or'), (20, 'note::'), (20, 'may'), (20, 'cluster'), (19, 'through'), (19, 'number'), (19, 'Make'), (19, 'information'), (18, 'It'), (18, 'must'), (18, '==========='), (18, '============='), (18, '===================='), (18, 'at'), (18, 'identifier'), (18, 'contains'), (18, 'which'), (18, 'file'), (17, 'use'), (17, 'machine'), (16, '.'), (16, 'following'), (16, 'up'

In [31]:
sc.stop()
spark.stop()